In [1]:
!pip install efficient_apriori
!pip install fptools

In [2]:
import pandas as pd
import numpy as np
from efficient_apriori import apriori as EA
from mlxtend.frequent_patterns import apriori, association_rules

## 加载数据：


In [3]:
df = pd.read_csv('./MarketBasket/Market_Basket_Optimisation.csv',header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,butter,light mayo,fresh bread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,burgers,frozen vegetables,eggs,french fries,magazines,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7499,escalope,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


使用efficient_apriori进行关联分析：

将数据存入transaction

In [5]:
用于存商品数据
# transactions = []

# for i in range(0,df.shape[0]):
#     temp = set()
#     for j in range(0,df.shape[1]):
#         item = str(df.values[i,j])
#         if item != 'nan':
#             temp.add(item)
#     transactions.append(temp)

### 订单去重后进行分析

In [7]:
# 测试第二行删掉空值后转为列表的效果
df.loc[1].dropna().to_list()

['burgers', 'meatballs', 'eggs']

In [10]:
# 等价于上一个for循环的代码
# 用于存商品数据
transactions = []

# 遍历每一行
for i in range(0, df.shape[0]):
    # 每一笔订单，外层增加了set()对订单中的商品进行去重
    temp = set(df.loc[i].dropna().to_list())
    transactions.append(temp)
    
transactions

## 计算频繁项集和关联规则

In [11]:
%%time
itemsets, rules = EA(transactions, min_support=0.04, min_confidence=0.2)

Wall time: 304 ms


In [12]:
print("频繁项集:\n",itemsets)
print("关联规则:\n",rules)

频繁项集:
 {1: {('olive oil',): 494, ('honey',): 356, ('salmon',): 319, ('shrimp',): 536, ('frozen smoothie',): 475, ('mineral water',): 1788, ('low fat yogurt',): 574, ('green tea',): 991, ('eggs',): 1348, ('burgers',): 654, ('turkey',): 469, ('milk',): 972, ('whole wheat rice',): 439, ('french fries',): 1282, ('soup',): 379, ('spaghetti',): 1306, ('frozen vegetables',): 715, ('cookies',): 603, ('cooking oil',): 383, ('champagne',): 351, ('chicken',): 450, ('chocolate',): 1229, ('tomatoes',): 513, ('pancakes',): 713, ('grated cheese',): 393, ('fresh bread',): 323, ('escalope',): 595, ('ground beef',): 737, ('herb & pepper',): 371, ('cake',): 608}, 2: {('milk', 'mineral water'): 360, ('eggs', 'mineral water'): 382, ('mineral water', 'spaghetti'): 448, ('ground beef', 'mineral water'): 307, ('chocolate', 'mineral water'): 395}}
关联规则:
 [{mineral water} -> {milk}, {milk} -> {mineral water}, {mineral water} -> {eggs}, {eggs} -> {mineral water}, {spaghetti} -> {mineral water}, {mineral water} -

In [13]:
#关联规则的数量有9个
len(rules)

9

看起来去重前后结果没有变化，在2.2中使用短数据集再验证下

In [14]:
#存在重复商品
df.loc[4494].dropna()

0            ham
1           eggs
2          honey
3           gums
4    light cream
5            ham
Name: 4494, dtype: object

In [15]:
#存在重复商品
df.loc[4394].dropna()

0             burgers
1                 ham
2                eggs
3    whole wheat rice
4                 ham
5        french fries
6             cookies
7           green tea
Name: 4394, dtype: object

测试订单中重复商品是否影响efficient_apriori关联分析结果：

In [16]:
transactions_ = [('eggs', 'bacon', 'soup', 'soup'),
                ('eggs', 'bacon', 'apple', 'bacon'),
                ('soup', 'bacon', 'banana')]
itemsets, rules = EA(transactions_,min_support=0.5,min_confidence=1)
print("频繁项集:\n",itemsets)
print("关联规则:\n",rules)

频繁项集:
 {1: {('eggs',): 2, ('soup',): 2, ('bacon',): 3}, 2: {('bacon', 'eggs'): 2, ('bacon', 'soup'): 2}}
关联规则:
 [{eggs} -> {bacon}, {soup} -> {bacon}]


#### 小结：通过这个例子可以看出，当同一个元组(也就是同一个订单)中存在重复商品时，不影响频繁项集的计数结果。因为它统计的是某商品在不同订单中出现的次数，而不是同一个订单中的出现次数。

# 使用mlxtend进行关联分析


## 尝试两种不同的df处理方法

## df处理方法1：列字段合并再进行读热编码

In [17]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 创建空DataFrame
#### 用于存储拼接后的字符串

In [18]:
df_new = pd.DataFrame(columns=['items'])
df_new

,items


### 将df每行中的商品拼接成一个字符串
#### 并存入新的DataFrame中

In [19]:
%%time
for i in range(df.shape[0]):
    df_new.loc[i] = df.loc[i].str.cat(sep='/')

Wall time: 14.9 s


In [20]:
df_new

,items
0,shrimp/almonds/avocado/vegetables mix/green gr...
1,burgers/meatballs/eggs
2,chutney
3,turkey/avocado
4,mineral water/milk/energy bar/whole wheat rice...
...,...
7496,butter/light mayo/fresh bread
7497,burgers/frozen vegetables/eggs/french fries/ma...
7498,chicken
7499,escalope/green tea


In [21]:
type(df_new.loc[0, 'items'])

str

## 对新DataFrame进行one-hot编码

In [22]:
one_hot_df = df_new['items'].str.get_dummies(sep="/")
one_hot_df

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# 所有列最大值的和=120，说明每一列最大值均为1，符合one-hot编码的特征
one_hot_df.max(axis=0).sum()

120

### 获取频繁项集通过one-hot编码的DataFrame对象


apriori(
    df,
    min_support=0.5,
    use_colnames=False,
    max_len=None,
    verbose=0,
    low_memory=False,
)

In [26]:
%%time
itemsets = apriori(one_hot_df,min_support=0.04,use_colnames=True)

Wall time: 74.8 ms


In [27]:
itemsets.sort_values(by=['support'], ascending=False)

,support,itemsets
20,0.238368,(mineral water)
7,0.179709,(eggs)
26,0.174110,(spaghetti)
9,0.170911,(french fries)
4,0.163845,(chocolate)
14,0.132116,(green tea)
19,0.129583,(milk)
15,0.098254,(ground beef)
12,0.095321,(frozen vegetables)
22,0.095054,(pancakes)


## 从频繁项集中找出符合条件的关联条件
 利用mlxtend进行数据关联分析，查看返回值各列含义

 association_rules(
    df,
    metric='confidence',
    min_threshold=0.8,
    support_only=False,
)

In [29]:
%%time
rules = association_rules(itemsets,metric='lift',min_threshold=1)

Wall time: 4.99 ms


In [30]:
#按规则提升度降序排序
rules.sort_values(by=['lift'],ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(mineral water),(ground beef),0.238368,0.098254,0.040928,0.171700,1.747522,0.017507,1.088672
5,(ground beef),(mineral water),0.098254,0.238368,0.040928,0.416554,1.747522,0.017507,1.305401
6,(mineral water),(milk),0.238368,0.129583,0.047994,0.201342,1.553774,0.017105,1.089850
7,(milk),(mineral water),0.129583,0.238368,0.047994,0.370370,1.553774,0.017105,1.209650
8,(mineral water),(spaghetti),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008
9,(spaghetti),(mineral water),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314
1,(chocolate),(mineral water),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357
0,(mineral water),(chocolate),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256
2,(eggs),(mineral water),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815
3,(mineral water),(eggs),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158


In [31]:
'mineral water' in one_hot_df.columns

True

## df处理方法2：直接整行读入CSV


#### 这种方法感觉更适合mlxtend包的apriori算法

### 数据加载

In [33]:
df = pd.read_csv('./MarketBasket/Market_Basket_Optimisation.csv', sep='\t',header=None)
df

,0
0,"shrimp,almonds,avocado,vegetables mix,green gr..."
1,"burgers,meatballs,eggs"
2,chutney
3,"turkey,avocado"
4,"mineral water,milk,energy bar,whole wheat rice..."
...,...
7496,"butter,light mayo,fresh bread"
7497,"burgers,frozen vegetables,eggs,french fries,ma..."
7498,chicken
7499,"escalope,green tea"


### df独热编码

In [34]:
%%time
df_one_hot = df[0].str.get_dummies(sep=',')

Wall time: 695 ms


In [35]:
df_one_hot

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
#校验是否最大值都是1
df_one_hot.max().sum()

120

### 计算频繁项集:

In [37]:
%%time
frequence_items = apriori(df_one_hot,min_support=0.04,use_colnames=True)

Wall time: 61.8 ms


In [38]:
%%time
frequence_items.sort_values(by=['support'],ascending=False)

Wall time: 997 µs


,support,itemsets
20,0.238368,(mineral water)
7,0.179709,(eggs)
26,0.174110,(spaghetti)
9,0.170911,(french fries)
4,0.163845,(chocolate)
14,0.132116,(green tea)
19,0.129583,(milk)
15,0.098254,(ground beef)
12,0.095321,(frozen vegetables)
22,0.095054,(pancakes)


### 根据频繁项集求关联规则

In [39]:
%%time
rules = association_rules(frequence_items,metric='lift',min_threshold=1)
rules.sort_values(by='lift',ascending=False)

Wall time: 3.99 ms


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(mineral water),(ground beef),0.238368,0.098254,0.040928,0.171700,1.747522,0.017507,1.088672
5,(ground beef),(mineral water),0.098254,0.238368,0.040928,0.416554,1.747522,0.017507,1.305401
6,(mineral water),(milk),0.238368,0.129583,0.047994,0.201342,1.553774,0.017105,1.089850
7,(milk),(mineral water),0.129583,0.238368,0.047994,0.370370,1.553774,0.017105,1.209650
8,(mineral water),(spaghetti),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008
9,(spaghetti),(mineral water),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314
1,(chocolate),(mineral water),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357
0,(mineral water),(chocolate),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256
2,(eggs),(mineral water),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815
3,(mineral water),(eggs),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158


## efficient_apriori和mlxtend的apriori算法比较：

### 除了efficient_apriori效率高，返回参数少；mlxtend效率低，返回参数多以外：

    1、efficient_apriori的apriori算法，频繁项集和关联规则是一起输出；且只能通过最小置信度来获取关联规则；
    2、mlxtend的apriori算法，提供2个方法，一个输出计算频繁项集，一个输出计算关联规则，支持最小置信度、最小支持度等8个指标来获取关联规则；
    3、mlxtend的apriori算法结果更加清晰直观；
    4、efficient_apriori的apriori算法，得到的关联规则中包含的支持度等信息是隐藏在Rule对象中的，需要单独打印Rule对象才能看到，不如mlxtend的结果直观。

# 尝试FPGrowth算法

## fptools包

In [40]:
import fptools as fp

In [42]:
transactions

In [43]:
%%time
tree = fp.build_tree(transactions,minsup=300)

Wall time: 39.9 ms


In [44]:
tree

(<fptools.FPTree at 0x2114bfede50>,
 {'salmon': 0,
  'fresh bread': 1,
  'champagne': 2,
  'honey': 3,
  'herb & pepper': 4,
  'soup': 5,
  'cooking oil': 6,
  'grated cheese': 7,
  'whole wheat rice': 8,
  'chicken': 9,
  'turkey': 10,
  'frozen smoothie': 11,
  'olive oil': 12,
  'tomatoes': 13,
  'shrimp': 14,
  'low fat yogurt': 15,
  'escalope': 16,
  'cookies': 17,
  'cake': 18,
  'burgers': 19,
  'pancakes': 20,
  'frozen vegetables': 21,
  'ground beef': 22,
  'milk': 23,
  'green tea': 24,
  'chocolate': 25,
  'french fries': 26,
  'spaghetti': 27,
  'eggs': 28,
  'mineral water': 29})

In [46]:
tree[0].nodes

In [47]:
len(tree[0].rank)

30

In [48]:
#项头表
tree[0].rank

{'salmon': 0,
 'fresh bread': 1,
 'champagne': 2,
 'honey': 3,
 'herb & pepper': 4,
 'soup': 5,
 'cooking oil': 6,
 'grated cheese': 7,
 'whole wheat rice': 8,
 'chicken': 9,
 'turkey': 10,
 'frozen smoothie': 11,
 'olive oil': 12,
 'tomatoes': 13,
 'shrimp': 14,
 'low fat yogurt': 15,
 'escalope': 16,
 'cookies': 17,
 'cake': 18,
 'burgers': 19,
 'pancakes': 20,
 'frozen vegetables': 21,
 'ground beef': 22,
 'milk': 23,
 'green tea': 24,
 'chocolate': 25,
 'french fries': 26,
 'spaghetti': 27,
 'eggs': 28,
 'mineral water': 29}

In [49]:
mineral_water_node = tree[0].nodes['mineral water'][0]
mineral_water_node

In [50]:
#子孩子节点
mineral_water_node.children

defaultdict(fptools.FPNode,
            {'green tea': <fptools.FPNode at 0x2114bfdff40>,
             'eggs': <fptools.FPNode at 0x2114bb133d0>,
             'salmon': <fptools.FPNode at 0x2114c5611f0>,
             'spaghetti': <fptools.FPNode at 0x2114c561850>,
             'ground beef': <fptools.FPNode at 0x2114c5641f0>,
             'cake': <fptools.FPNode at 0x2114c564250>,
             'chicken': <fptools.FPNode at 0x2114c564670>,
             'chocolate': <fptools.FPNode at 0x2114c5671f0>,
             'french fries': <fptools.FPNode at 0x2114c5675b0>,
             'olive oil': <fptools.FPNode at 0x2114c56d130>,
             'frozen vegetables': <fptools.FPNode at 0x2114c562130>,
             'turkey': <fptools.FPNode at 0x2114c562a90>,
             'shrimp': <fptools.FPNode at 0x2114c562e50>,
             'fresh bread': <fptools.FPNode at 0x2114c57d970>,
             'frozen smoothie': <fptools.FPNode at 0x2114c101790>,
             'honey': <fptools.FPNode at 0x2114c106550>,


In [51]:
mineral_water_node.count

1788

In [52]:
mineral_water_node.item

'mineral water'

In [53]:
green_tea_node = mineral_water_node.children['green tea']
green_tea_node

In [54]:
green_tea_node.count

78

In [55]:
green_tea_node.parent.item

'mineral water'

In [56]:
%%time
items = [i for i in fp.fpgrowth(tree[0], 3000)]

Wall time: 18 ms


In [57]:
print(len(items))
items

30


[['mineral water'],
 ['green tea'],
 ['low fat yogurt'],
 ['shrimp'],
 ['olive oil'],
 ['frozen smoothie'],
 ['honey'],
 ['salmon'],
 ['eggs'],
 ['burgers'],
 ['turkey'],
 ['milk'],
 ['whole wheat rice'],
 ['french fries'],
 ['soup'],
 ['spaghetti'],
 ['frozen vegetables'],
 ['cookies'],
 ['cooking oil'],
 ['champagne'],
 ['chocolate'],
 ['chicken'],
 ['tomatoes'],
 ['pancakes'],
 ['grated cheese'],
 ['fresh bread'],
 ['ground beef'],
 ['escalope'],
 ['herb & pepper'],
 ['cake']]

In [58]:
#最小支持频数统计频繁项集， 返回生成器
generate = fp.frequent_itemsets(transactions,minsup=300)
generate

<generator object frequent_itemsets at 0x000002114C1CD9E0>

In [60]:
itemsets = [s for s in generate]
print(len(itemsets))
itemsets

35


[['mineral water'],
 ['green tea'],
 ['low fat yogurt'],
 ['shrimp'],
 ['olive oil'],
 ['frozen smoothie'],
 ['honey'],
 ['salmon'],
 ['eggs'],
 ['eggs', 'mineral water'],
 ['burgers'],
 ['turkey'],
 ['milk'],
 ['milk', 'mineral water'],
 ['whole wheat rice'],
 ['french fries'],
 ['soup'],
 ['spaghetti'],
 ['spaghetti', 'mineral water'],
 ['frozen vegetables'],
 ['cookies'],
 ['cooking oil'],
 ['champagne'],
 ['chocolate'],
 ['chocolate', 'mineral water'],
 ['chicken'],
 ['tomatoes'],
 ['pancakes'],
 ['grated cheese'],
 ['fresh bread'],
 ['ground beef'],
 ['ground beef', 'mineral water'],
 ['escalope'],
 ['herb & pepper'],
 ['cake']]

In [61]:
type(itemsets[0][0])

str

In [62]:
one_hot_df.sum(axis=0).sort_values(ascending=False)[:30]

mineral water        1788
eggs                 1348
spaghetti            1306
french fries         1282
chocolate            1229
green tea             991
milk                  972
ground beef           737
frozen vegetables     715
pancakes              713
burgers               654
cake                  608
cookies               603
escalope              595
low fat yogurt        574
shrimp                536
tomatoes              513
olive oil             494
frozen smoothie       475
turkey                469
chicken               450
whole wheat rice      439
grated cheese         393
cooking oil           383
soup                  379
herb & pepper         371
honey                 356
champagne             351
fresh bread           323
salmon                319
dtype: int64

#### 小结：FPGrowth的算法内，只找到支持按照最小支持度来求频繁项集的方法。计算效率确实快。它的最小支持度是最小出现次数，和前面两种算法的最小出现概率不同。另外就是它得到的频繁项集中不会显示数据。